# AWS CLI Mate example

First, assuming that you have one default AWS CLI profile and three more profiles ``p1``, ``p2``, ``p3``, you want to use ``awscli_mate`` to manage your AWS CLI profiles.

In [36]:
import os
from pathlib import Path

dir_here = Path(os.getcwd())
path_config = dir_here / "tests" / "home" / ".aws" / "config"
path_credentials = dir_here / "tests" / "home" / ".aws" / "credentials"

In [37]:
print(path_config.read_text())

[default]
region = us-west-1
output = json

#--- begin of organization ---
#--- p1 ---
[profile p1]
# comment for p1
region = us-east-1
output = json
#--- p1 ---
#
#--- p2 ---
[profile p2]
region = us-east-2
output = json
role_arn = arn:aws:iam::111122223333:role/fake-role-name
source_profile = p1 # comment for p2
#--- p2 ---
#
#--- p3 ---
[profile p3]
region = us-east-3
output = json
#--- p3 ---
#--- end of organization ---



In [38]:
print(path_credentials.read_text())

[default]
aws_access_key_id = ZZZ
aws_secret_access_key = ZZZ

[p1]
# comment for p1
aws_access_key_id = AAA
aws_secret_access_key = AAA

[p2]

[p3]
aws_access_key_id = CCC
aws_secret_access_key = CCC
aws_session_token = CCC # comment for p3



Now let's manipulate AWS CLI config and credentials files.

In [39]:
from awscli_mate import AWSCliConfig

awscli_config = AWSCliConfig(path_config=path_config, path_credentials=path_credentials)
# AWSCliConfig() # do this if you want to use the ~/.aws/config and ~/.aws/credentials

config, credentials = awscli_config.read_config()

In [40]:
import json

def jsonify_config(config):
    data = dict()
    for section_name, section_data in config.items():
        data[section_name] = dict(section_data)
    return json.dumps(data, indent=4)

print(jsonify_config(config))

{
    "DEFAULT": {},
    "default": {
        "region": "us-west-1",
        "output": "json"
    },
    "profile p1": {
        "region": "us-east-1",
        "output": "json"
    },
    "profile p2": {
        "region": "us-east-2",
        "output": "json",
        "role_arn": "arn:aws:iam::111122223333:role/fake-role-name",
        "source_profile": "p1 # comment for p2"
    },
    "profile p3": {
        "region": "us-east-3",
        "output": "json"
    }
}


In [41]:
awscli_config.replace_section_data(
    config, from_section_name="profile p1", to_section_name="default"
)
print(jsonify_config(config))

{
    "DEFAULT": {},
    "default": {
        "region": "us-east-1",
        "output": "json"
    },
    "profile p1": {
        "region": "us-east-1",
        "output": "json"
    },
    "profile p2": {
        "region": "us-east-2",
        "output": "json",
        "role_arn": "arn:aws:iam::111122223333:role/fake-role-name",
        "source_profile": "p1 # comment for p2"
    },
    "profile p3": {
        "region": "us-east-3",
        "output": "json"
    }
}


In [42]:
print(jsonify_config(credentials))

{
    "DEFAULT": {},
    "default": {
        "aws_access_key_id": "ZZZ",
        "aws_secret_access_key": "ZZZ"
    },
    "p1": {
        "aws_access_key_id": "AAA",
        "aws_secret_access_key": "AAA"
    },
    "p2": {},
    "p3": {
        "aws_access_key_id": "CCC",
        "aws_secret_access_key": "CCC",
        "aws_session_token": "CCC # comment for p3"
    }
}


In [43]:
awscli_config.replace_section_data(
    credentials, from_section_name="p1", to_section_name="default"
)
print(jsonify_config(credentials))

{
    "DEFAULT": {},
    "default": {
        "aws_access_key_id": "AAA",
        "aws_secret_access_key": "AAA"
    },
    "p1": {
        "aws_access_key_id": "AAA",
        "aws_secret_access_key": "AAA"
    },
    "p2": {},
    "p3": {
        "aws_access_key_id": "CCC",
        "aws_secret_access_key": "CCC",
        "aws_session_token": "CCC # comment for p3"
    }
}
